In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from astropy.constants import G
import astropy.units as u
import warnings
import traceback
from colossus.cosmology import cosmology
import symlib
import csv

In [5]:
def plot_ppsd_profiles_colored_by_accretion(base_dir, suite_name):

    # Directories
    profile_dir = os.path.join(base_dir, "output", suite_name, "ppsd_profiles")
    output_dir = os.path.join(base_dir, "output", suite_name, "figures")
    gamma_file = os.path.join(base_dir, "output", suite_name, "accretion_rates.csv")
    os.makedirs(output_dir, exist_ok=True)

    # Load accretion rate table
    df_gamma = pd.read_csv(gamma_file)
    gamma_dict = dict(zip(df_gamma["halo_index"], df_gamma["gamma"]))

    # Load profile data
    files = sorted([f for f in os.listdir(profile_dir) if f.endswith(".csv")])
    ppsd_r, ppsd_tot, mass_profiles, accretion_rates = [], [], [], []

    for f in files:
        df = pd.read_csv(os.path.join(profile_dir, f))
        r = df["r_scaled"].values
        Q_r = df["Q_r"].values
        Q_tot = df["Q_tot"].values
        m = df["m_scaled"].values
        ppsd_r.append(Q_r)
        ppsd_tot.append(Q_tot)
        mass_profiles.append(m)

        halo_id = int(f.split("_")[1])
        gamma = gamma_dict.get(halo_id, np.nan)
        accretion_rates.append(gamma)

    ppsd_r = np.array(ppsd_r)
    ppsd_tot = np.array(ppsd_tot)
    mass_profiles = np.array(mass_profiles)
    accretion_rates = np.array(accretion_rates)
    n_halos = len(ppsd_r)
    r_global = r  # All halo profiles assumed to share the same radial bins

    # Reference curves
    mean_Qr = np.nanmean(ppsd_r, axis=0)
    mean_Qtot = np.nanmean(ppsd_tot, axis=0)
    log_r = np.log(r_global)
    A_r = np.exp(np.mean(np.log(mean_Qr) + 1.875 * log_r))
    A_tot = np.exp(np.mean(np.log(mean_Qtot) + 1.875 * log_r))
    ref_curve_r = A_r * r_global**(-1.875)
    ref_curve_tot = A_tot * r_global**(-1.875)

    # Residuals
    residuals_r, residuals_tot = [], []
    for i in range(n_halos):
        res_r = np.log10(ppsd_r[i]) - np.log10(ref_curve_r)
        res_t = np.log10(ppsd_tot[i]) - np.log10(ref_curve_tot)
        residuals_r.append(res_r)
        residuals_tot.append(res_t)
    residuals_r = np.array(residuals_r)
    residuals_tot = np.array(residuals_tot)
    mean_res_r = np.nanmean(residuals_r, axis=0)
    std_res_r = np.nanstd(residuals_r, axis=0)
    mean_res_tot = np.nanmean(residuals_tot, axis=0)
    std_res_tot = np.nanstd(residuals_tot, axis=0)

    # Colormap for accretion rate
    cmap = cm.plasma
    log_gamma = np.log10(accretion_rates)
    vmin = np.nanpercentile(log_gamma, 5)
    vmax = np.nanpercentile(log_gamma, 95)
    norm = plt.Normalize(vmin, vmax)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    # --- Figure 1: Q vs r ---
    fig1, ax1 = plt.subplots(1, 2, figsize=(14, 5), dpi=500, constrained_layout=True)
    for i in range(n_halos):
        color = 'gray' if np.isnan(log_gamma[i]) else cmap(norm(log_gamma[i]))
        ax1[0].plot(r_global, ppsd_r[i], color=color, lw=0.7)
        ax1[1].plot(r_global, ppsd_tot[i], color=color, lw=0.7)
    ax1[0].set(title=r"$Q_r$ vs $r$", xscale="log", yscale="log", xlabel=r"$r / r_{\rm vir}$", ylabel=r"$Q_r$")
    ax1[1].set(title=r"$Q_{\rm tot}$ vs $r$", xscale="log", yscale="log", xlabel=r"$r / r_{\rm vir}$", ylabel=r"$Q_{\rm tot}$")
    for ax in ax1: ax.grid(True, which="both", linestyle=":")
    fig1.colorbar(sm, ax=ax1.ravel().tolist(), shrink=0.9).set_label(r"$\log_{10}(\Gamma)$ [M$_\odot$/Gyr]")
    fig1.suptitle(f"PPSD vs r (Colored by Accretion Rate): {suite_name}")
    fig1.savefig(os.path.join(output_dir, "PPSD_vs_r (colored by accretion).png"))
    plt.show()
    plt.close(fig1)

    # --- Figure 2: Q vs M(<r) ---
    fig2, ax2 = plt.subplots(1, 2, figsize=(14, 5), dpi=500, constrained_layout=True)
    for i in range(n_halos):
        color = 'gray' if np.isnan(log_gamma[i]) else cmap(norm(log_gamma[i]))
        ax2[0].plot(mass_profiles[i], ppsd_r[i], color=color, lw=0.7)
        ax2[1].plot(mass_profiles[i], ppsd_tot[i], color=color, lw=0.7)
    ax2[0].set(title=r"$Q_r$ vs $M(<r)$", xscale="log", yscale="log", xlabel=r"$M(<r)/M_{\rm vir}$", ylabel=r"$Q_r$")
    ax2[1].set(title=r"$Q_{\rm tot}$ vs $M(<r)$", xscale="log", yscale="log", xlabel=r"$M(<r)/M_{\rm vir}$", ylabel=r"$Q_{\rm tot}$")
    for ax in ax2: ax.grid(True, which="both", linestyle=":")
    fig2.colorbar(sm, ax=ax2.ravel().tolist(), shrink=0.9).set_label(r"$\log_{10}(\Gamma)$ [M$_\odot$/Gyr]")
    fig2.suptitle(f"PPSD vs Mass (Colored by Accretion Rate): {suite_name}")
    fig2.savefig(os.path.join(output_dir, "PPSD_vs_mass (colored by accretion).png"))
    plt.show()
    plt.close(fig2)

    # --- Figure 3: Residuals ---
    fig3, ax3 = plt.subplots(1, 2, figsize=(14, 5), dpi=500, constrained_layout=True)
    for i in range(n_halos):
        color = 'gray' if np.isnan(log_gamma[i]) else cmap(norm(log_gamma[i]))
        ax3[0].plot(r_global, residuals_r[i], color=color, lw=0.7)
        ax3[1].plot(r_global, residuals_tot[i], color=color, lw=0.7)
    ax3[0].plot(r_global, mean_res_r, 'k-', lw=1)
    ax3[0].fill_between(r_global, mean_res_r - std_res_r, mean_res_r + std_res_r, color='gray', alpha=0.3)
    ax3[0].axhline(0, color='r', linestyle='--', lw=1)
    ax3[0].set(title=r"$\log_{10}(Q_r/Q_{\rm ref})$ vs $r / r_{\rm vir}$", xlabel=r"$r / r_{\rm vir}$", ylabel=r"$\log_{10}(Q_r/Q_{\rm ref})$", xscale="log", ylim=(-1.5, 1.5))
    ax3[1].plot(r_global, mean_res_tot, 'k-', lw=1)
    ax3[1].fill_between(r_global, mean_res_tot - std_res_tot, mean_res_tot + std_res_tot, color='gray', alpha=0.3)
    ax3[1].axhline(0, color='r', linestyle='--', lw=1)
    ax3[1].set(title=r"$\log_{10}(Q_{\rm tot}/Q_{\rm ref})$ vs $r / r_{\rm vir}$", xlabel=r"$r / r_{\rm vir}$", ylabel=r"$\log_{10}(Q_{\rm tot}/Q_{\rm ref})$", xscale="log", ylim=(-1.5, 1.5))
    for ax in ax3: ax.grid(True, which="both", linestyle=":")
    fig3.colorbar(sm, ax=ax3.ravel().tolist(), shrink=0.9).set_label(r"$\log_{10}(\Gamma)$ [M$_\odot$/Gyr]")
    fig3.suptitle(f"PPSD Residuals (Colored by Accretion Rate): {suite_name}")
    fig3.savefig(os.path.join(output_dir, "PPSD_residuals (colored by accretion).png"))
    plt.show()
    plt.close(fig3)

    print(f"[Saved] Accretion-rate-colored PPSD figures for {suite_name}")

In [ ]:
base_dir = "/Users/fengbocheng/Projects/Symphony-PPSD"
suite_names = [
    "SymphonyCluster",
]

for suite in suite_names:
    plot_ppsd_profiles_colored_by_accretion(base_dir, suite)

In [8]:
from scipy.interpolate import interp1d

def plot_ppsd_slope_colored_by_accretion(base_dir, suite_name):
    # --- Path Setup ---
    slope_r_dir = os.path.join(base_dir, "output", suite_name, "ppsd_slope_profiles_r")
    slope_m_dir = os.path.join(base_dir, "output", suite_name, "ppsd_slope_profiles_m")
    output_dir = os.path.join(base_dir, "output", suite_name, "figures")
    accretion_path = os.path.join(base_dir, "output", suite_name, "accretion_rates.csv")
    os.makedirs(output_dir, exist_ok=True)

    slope_r_files = sorted([f for f in os.listdir(slope_r_dir) if f.endswith(".csv")])
    slope_m_files = sorted([f for f in os.listdir(slope_m_dir) if f.endswith(".csv")])
    n_halos = min(len(slope_r_files), len(slope_m_files))

    # --- Load accretion rates ---
    df_gamma = pd.read_csv(accretion_path)
    gamma_dict = dict(zip(df_gamma["halo_index"], df_gamma["gamma"]))

    gammas = []
    r_list, m_list = [], []
    slope_Q_tot_r_list, slope_Q_rad_r_list = [], []
    slope_Q_tot_m_list, slope_Q_rad_m_list = [], []

    for i in range(n_halos):
        halo_id = int(slope_r_files[i].split("_")[1])
        gamma = gamma_dict.get(halo_id, np.nan)
        gammas.append(gamma)

        try:
            df_r = pd.read_csv(os.path.join(slope_r_dir, slope_r_files[i]))
            df_m = pd.read_csv(os.path.join(slope_m_dir, slope_m_files[i]))
        except Exception as e:
            print(f"[Warning] Failed to load halo {halo_id}: {e}")
            continue

        try:
            r = df_r["r_scaled"].values
            m = df_m["m_scaled"].values
            slope_Q_tot_r = df_r["slope_Q_tot"].values
            slope_Q_rad_r = df_r["slope_Q_r"].values
            slope_Q_tot_m = df_m["slope_Q_tot"].values
            slope_Q_rad_m = df_m["slope_Q_r"].values

            r_list.append(r)
            m_list.append(m)
            slope_Q_tot_r_list.append(slope_Q_tot_r)
            slope_Q_rad_r_list.append(slope_Q_rad_r)
            slope_Q_tot_m_list.append(slope_Q_tot_m)
            slope_Q_rad_m_list.append(slope_Q_rad_m)
        except Exception as e:
            print(f"[Warning] Plotting failed for halo {halo_id}: {e}")
            continue

    # --- Color setup ---
    log_gamma = np.log10(np.array(gammas))
    vmin, vmax = np.nanpercentile(log_gamma, [5, 95])
    cmap = cm.plasma
    norm = colors.Normalize(vmin=vmin, vmax=vmax)
    sm = cm.ScalarMappable(norm=norm, cmap=cmap)

    # --- Plotting setup ---
    fig1, axes1 = plt.subplots(1, 2, figsize=(13, 5), dpi=500, constrained_layout=True)
    fig2, axes2 = plt.subplots(1, 2, figsize=(13, 5), dpi=500, constrained_layout=True)
    ax_slope_tot, ax_slope_rad = axes1
    ax_slope_tot_m, ax_slope_rad_m = axes2

    for r, m, s_tr, s_rr, s_tm, s_rm, g in zip(r_list, m_list, slope_Q_tot_r_list, slope_Q_rad_r_list,
                                              slope_Q_tot_m_list, slope_Q_rad_m_list, log_gamma):
        color = cmap(norm(g)) if np.isfinite(g) else "gray"
        ax_slope_tot.plot(r, s_tr, color=color, alpha=0.5, lw=0.7)
        ax_slope_rad.plot(r, s_rr, color=color, alpha=0.5, lw=0.7)
        ax_slope_tot_m.plot(m, s_tm, color=color, alpha=0.5, lw=0.7)
        ax_slope_rad_m.plot(m, s_rm, color=color, alpha=0.5, lw=0.7)

    # --- Mean & Std interpolation ---
    def plot_mean_std_interp(ax, x_list, y_list, x_range, label="Mean", color="black"):
        x_common = np.logspace(np.log10(x_range[0]), np.log10(x_range[1]), 200)
        y_interp_list = []

        for x, y in zip(x_list, y_list):
            mask = np.isfinite(x) & np.isfinite(y)
            if np.sum(mask) < 2:
                continue
            try:
                f_interp = interp1d(x[mask], y[mask], bounds_error=False, fill_value=np.nan)
                y_interp = f_interp(x_common)
                y_interp_list.append(y_interp)
            except:
                continue

        if len(y_interp_list) == 0:
            return

        y_array = np.array(y_interp_list)
        y_mean = np.nanmean(y_array, axis=0)
        y_std = np.nanstd(y_array, axis=0)

        ax.plot(x_common, y_mean, color=color, lw=0.8, label=label)
        ax.fill_between(x_common, y_mean - y_std, y_mean + y_std, color=color, alpha=0.2)

    # --- Compute interpolation range ---
    r_all = np.concatenate([r for r in r_list if np.all(np.isfinite(r))])
    m_all = np.concatenate([m for m in m_list if np.all(np.isfinite(m))])
    r_range = [np.nanpercentile(r_all, 5), np.nanpercentile(r_all, 95)]
    m_range = [np.nanpercentile(m_all, 5), np.nanpercentile(m_all, 95)]

    # --- Interpolated mean lines ---
    plot_mean_std_interp(ax_slope_tot, r_list, slope_Q_tot_r_list, r_range, r"Mean $d\log Q / d\log r$")
    plot_mean_std_interp(ax_slope_rad, r_list, slope_Q_rad_r_list, r_range, r"Mean $d\log Q_r / d\log r$")
    plot_mean_std_interp(ax_slope_tot_m, m_list, slope_Q_tot_m_list, m_range, r"Mean $d\log Q / d\log M$")
    plot_mean_std_interp(ax_slope_rad_m, m_list, slope_Q_rad_m_list, m_range, r"Mean $d\log Q_r / d\log M$")

    # --- Axis formatting ---
    for ax, label in zip([ax_slope_tot, ax_slope_rad],
                         [r"$\mathrm{d}\log Q / \mathrm{d}\log r$", r"$\mathrm{d}\log Q_r / \mathrm{d}\log r$"]):
        ax.set_xlabel(r"$r / R_\mathrm{vir}$")
        ax.set_ylabel(label)
        ax.set_xscale("log")
        ax.set_ylim(-4, 1)
        ax.axhline(-1.875, color='k', ls='--', lw=0.7)
        ax.grid(True, which="both", linestyle=":")
        ax.legend()

    for ax, label in zip([ax_slope_tot_m, ax_slope_rad_m],
                         [r"$\mathrm{d}\log Q / \mathrm{d}\log M$", r"$\mathrm{d}\log Q_r / \mathrm{d}\log M$"]):
        ax.set_xlabel(r"$M(<r) / M_\mathrm{vir}$")
        ax.set_ylabel(label)
        ax.set_xscale("log")
        ax.set_ylim(-4, 1)
        ax.grid(True, which="both", linestyle=":")
        ax.legend()

    # --- Colorbar and Save ---
    fig1.colorbar(sm, ax=axes1.ravel().tolist(), shrink=0.9).set_label(r"$\log_{10}(\Gamma)$ [M$_\odot$/Gyr]")
    fig2.colorbar(sm, ax=axes2.ravel().tolist(), shrink=0.9).set_label(r"$\log_{10}(\Gamma)$ [M$_\odot$/Gyr]")

    fig1.suptitle(f"{suite_name} PPSD Slopes vs Radius colored by Accretion Rate", y=1.05)
    fig2.suptitle(f"{suite_name} PPSD Slopes vs Mass colored by Accretion Rate", y=1.05)

    try:
        fig1.savefig(os.path.join(output_dir, "PPSD_slopes_r_colored_by_accretion.png"))
        fig2.savefig(os.path.join(output_dir, "PPSD_slopes_m_colored_by_accretion.png"))
        print("[Success] Saved interpolated slope plots with mean and 1σ.")
        plt.show()
    except Exception as e:
        print(f"[Error] Failed to save figures: {e}")

    plt.close(fig1)
    plt.close(fig2)

In [ ]:
base_dir = "/Users/fengbocheng/Projects/Symphony-PPSD"
suite_names = [
    "SymphonyCluster",
]

for suite in suite_names:
    plot_ppsd_slope_colored_by_accretion(base_dir, suite)